In [ ]:
import asyncio
import aiohttp
from azure.iot.device.aio import IoTHubDeviceClient
from azure.iot.device import Message
import json
import random
from datetime import datetime, timedelta

Replace with your IoT Hub connection string

In [ ]:
CONNECTION_STRING = "redacted"

Define room types and their characteristics

In [ ]:
ROOM_TYPES = {
    "spark_suite": {
        "temp_range": (20, 23),
        "humidity_range": (40, 50),
        "energy_range": (100, 200),
        "water_range": (0, 5),
        "co2_range": (400, 1000),
        "co_range": (0, 5),
    },
    "maker_space": {
        "temp_range": (18, 25),
        "humidity_range": (30, 60),
        "energy_range": (200, 400),
        "water_range": (5, 20),
        "co2_range": (400, 1200),
        "co_range": (0, 10),
    },
    "robotics": {
        "temp_range": (18, 28),
        "humidity_range": (30, 70),
        "energy_range": (300, 600),
        "water_range": (0, 10),
        "co2_range": (400, 1500),
        "co_range": (0, 15),
    },
    "cafe": {
        "temp_range": (20, 25),
        "humidity_range": (40, 60),
        "energy_range": (200, 500),
        "water_range": (50, 200),
        "co2_range": (400, 1000),
        "co_range": (0, 5),
    },
    "bathroom": {
        "temp_range": (20, 24),
        "humidity_range": (50, 70),
        "energy_range": (50, 150),
        "water_range": (100, 500),
        "co2_range": (400, 800),
        "co_range": (0, 2),
    },
    "gmu_room": {
        "temp_range": (20, 24),
        "humidity_range": (40, 60),
        "energy_range": (150, 300),
        "water_range": (0, 10),
        "co2_range": (400, 1200),
        "co_range": (0, 5),
    },
    "hallway": {
        "temp_range": (18, 26),
        "humidity_range": (30, 60),
        "energy_range": (50, 100),
        "water_range": (0, 1),
        "co2_range": (400, 800),
        "co_range": (0, 2),
    },
}

Define rooms on the 3rd floor with updated occupancy limits

In [ ]:
ROOMS = {
    "Spark_Suite_A": {"type": "spark_suite", "size": 1730, "max_occupancy": 60},
    "Spark_Suite_B": {"type": "spark_suite", "size": 1874, "max_occupancy": 65},
    "Spark_Suite_C": {"type": "spark_suite", "size": 1939, "max_occupancy": 65},
    "Spark_Suite_D": {"type": "spark_suite", "size": 3246, "max_occupancy": 100},
    "Spark_Suite_E": {"type": "spark_suite", "size": 1758, "max_occupancy": 60},
    "Maker_Space": {"type": "maker_space", "size": 2000, "max_occupancy": 50},
    "Robotics_Assembly": {"type": "robotics", "size": 1500, "max_occupancy": 50},
    "Robotics_High_Bay": {"type": "robotics", "size": 2000, "max_occupancy": 50},
    "Cafe": {"type": "cafe", "size": 1000, "max_occupancy": 30},
    "GMU_Room_1": {"type": "gmu_room", "size": 2000, "max_occupancy": 75},
    "GMU_Room_2": {"type": "gmu_room", "size": 2000, "max_occupancy": 75},
    "Bathroom_1": {"type": "bathroom", "size": 100, "max_occupancy": 5},
    "Bathroom_2": {"type": "bathroom", "size": 100, "max_occupancy": 5},
    "Hallway_1": {"type": "hallway", "size": 500, "max_occupancy": None},
    "Hallway_2": {"type": "hallway", "size": 500, "max_occupancy": None},
}

In [ ]:
def generate_room_data(room_name, room_info, current_time):
    room_type = ROOM_TYPES[room_info['type']]
    size_factor = room_info['size'] / 1000  # Adjust values based on room size

    # Time-based factors
    hour = current_time.hour
    weekday = current_time.weekday() < 5  # True if weekday, False if weekend

    # Base occupancy
    if room_info['max_occupancy'] is not None:
        if weekday and 9 <= hour < 17:
            occupancy = random.randint(0, int(room_info['max_occupancy'] * 0.8))
        elif hour < 6 or hour >= 22:
            occupancy = random.randint(0, int(room_info['max_occupancy'] * 0.1))
        else:
            occupancy = random.randint(0, int(room_info['max_occupancy'] * 0.5))
    else:
        occupancy = 0  # For hallways and areas with null occupancy

    # Adjust metrics based on occupancy and time 
    temp = random.uniform(*room_type['temp_range']) + (occupancy * 0.05)
    humidity = random.uniform(*room_type['humidity_range']) + (occupancy * 0.1)
    energy_usage = random.uniform(*room_type['energy_range']) * size_factor * (0.5 + (occupancy / (room_info['max_occupancy'] or 1)))
    water_usage = random.uniform(*room_type['water_range']) * size_factor * (0.5 + (occupancy / (room_info['max_occupancy'] or 1)))
    co2 = random.uniform(*room_type['co2_range']) + (occupancy * 10)
    co = random.uniform(*room_type['co_range'])
    oxygen = 20.95 - (co2 - 400) / 10000  # Approximate oxygen level based on CO2

    # Special rules for different room types
    if room_info['type'] == 'cafe' and 11 <= hour < 14:
        occupancy = min(occupancy * 2, room_info['max_occupancy'])  # Busier during lunch hours
        energy_usage *= 1.3  # More energy use for food preparation
        water_usage *= 1.5  # More water use during busy hours
    elif room_info['type'] == 'robotics' and 9 <= hour < 18:
        energy_usage *= 1.5  # Higher energy use during testing hours
    elif room_info['type'] == 'bathroom':
        water_usage *= 1 + (random.random() * 2)  # Random spikes in water usage for bathrooms
    return {
        "deviceId": f"room_{room_name}",
        "timestamp": current_time.isoformat(),
        "temperature": round(temp, 2),
        "humidity": round(humidity, 2),
        "oxygen": round(oxygen, 2),
        "carbon_monoxide": round(co, 2),
        "carbon_dioxide": round(co2, 2),
        "energy_usage": round(energy_usage, 2),
        "water_usage": round(water_usage, 2),
        "occupancy": occupancy,
        "room_type": room_info['type'],
        "room_size": room_info['size'],
        "max_occupancy": room_info['max_occupancy']
    }

In [ ]:
async def send_device_data(device_client, data):
    msg = Message(json.dumps(data))
    msg.content_encoding = "utf-8"
    msg.content_type = "application/json"
    await device_client.send_message(msg)
    print(f"Message sent: {data}")

In [ ]:
async def main():
    device_client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)
    await device_client.connect()
    start_time = datetime.now()
    simulation_duration = timedelta(days=7)  # Simulate for 7 days
    try:
        while datetime.now() - start_time < simulation_duration:
            current_time = datetime.now()
            for room_name, room_info in ROOMS.items():
                data = generate_room_data(room_name, room_info, current_time)
                await send_device_data(device_client, data)
            await asyncio.sleep(300)  # Wait for 5 minutes before next data generation
    except KeyboardInterrupt:
        print("Simulation stopped by user")
    finally:
        await device_client.disconnect()

In [ ]:
if __name__ == "__main__":
    asyncio.run(main())